
# Creating entangled states with optimized circuits for particular backend


## Introduction

Explanation

## Idea

...

## Solution

In [40]:
## Importing the necesary libraries

In [41]:
from qiskit import QuantumCircuit
from qiskit.providers.fake_provider import FakePrague, FakeSherbrooke, FakeAuckland , FakeQuitoV2
from qiskit import transpile
from qiskit.tools.visualization import plot_histogram
from qiskit.quantum_info import entropy, Statevector
from rustworkx.visualization import graphviz_draw
import rustworkx as rw
from rustworkx import minimum_spanning_edges, PyGraph, dijkstra_shortest_path_lengths
import matplotlib.pyplot as plt
import math
from random import randrange
from IPython import get_ipython

## Naive Approach

We will take the backend with a semnificative number of qubits, in order to have a statistically significant result.

In [42]:
my_variables = set(dir())


# Get a fake backend from the fake provider
prague = FakePrague()
sherbrooke = FakeSherbrooke()
auckland = FakeAuckland()
quitov2 = FakeQuitoV2()


# Create a simple circuit
num_qubits_sherbrooke = sherbrooke.num_qubits
circuit = QuantumCircuit(num_qubits_sherbrooke)
circuit.h(0)
for i in range(num_qubits_sherbrooke-1):
    circuit.cx(i,i+1)

# this circuit is way too large to be shown in a figure
circuit.measure_all()
# circuit.draw()


naive_depth_sherbrooke = circuit.depth()
print(naive_depth_sherbrooke)




128


# Solution

In [43]:
def get_optimized_state_random(backend):
    findme = 'num_qubits'
    if findme in dir(backend):
        num_qubits = backend.num_qubits
        connections = backend.coupling_map
    else:
        num_qubits = backend.configuration().n_qubits
        connections = backend.configuration().coupling_map
    
    # Transpile the ideal circuit to a circuit that can be directly executed by the backend
    qc = QuantumCircuit(num_qubits)
    transpiled_circuit = transpile(qc, backend)
    transpiled_circuit.draw('mpl')

    # Run the transpiled circuit using the simulated fake backend
    job = backend.run(transpiled_circuit)
    # counts = job.result().get_counts()
    # plot_histogram(counts)

    targeted = {}
    for i in range(num_qubits):
        targeted.update({i: 'n'})
    start = randrange(num_qubits)
    print("Start qubit: ", start)

    qc.h(start)
    # print(qc.draw())
    # mark this qubit as 'targeted'
    targeted[start] = 't'

    list_ctrl = []
    list_ctrl.append(start)
    # print("Initial list of control qubits: ", list_ctrl)
    # print("First element of the list: ", list_ctrl[0])

    def non_entangled_exist(target_dict):
        ret = False
        for i in range(len(target_dict)):
            if target_dict[i] == 'n':
                ret = True

        return ret

    iteration = 0
    while non_entangled_exist(targeted):
        iteration += 1
        # helper list_ctrl to be used for swap at the end of the loop
        swp_list_ctrl = []
        # for each node in list_ctrl find connected nodes which haven't been entangled (targeted)
        for i in list_ctrl:
            # find connected nodes in connections not targeted
            for j in connections:
                # apply cx on it
                # mark it as targeted
                # add it to swp_list_ctrl
                if j[0] == i:
                    if targeted[j[1]] == 'n':
                        qc.cx(i, j[1])
                        targeted[j[1]] = 't'
                        swp_list_ctrl.append(j[1])
                if j[1] == i:
                    if targeted[j[0]] == 'n':
                        qc.cx(i, j[0])
                        targeted[j[0]] = 't'
                        swp_list_ctrl.append(j[0])
        # clear list_ctrl
        list_ctrl = []
        # replace list_ctrl with swp_list_ctrl
        for i in swp_list_ctrl:
            list_ctrl.append(i)
        qc.barrier()


    return [qc.depth(), num_qubits]

# Testing the solution on the Sherbrooke backend with a random initial state

print("The circuit depth: ", get_optimized_state_random(sherbrooke)[0])

Start qubit:  97
The circuit depth:  33


## Selecting a the qubit corresponding to the root node in the spanning tree

In [44]:
def get_optimized_state_root(backend):
    findme = 'num_qubits'
    if findme in dir(backend):
        num_qubits = backend.num_qubits
        connections = backend.coupling_map
    else:
        num_qubits = backend.configuration().n_qubits
        connections = backend.configuration().coupling_map
    
    # Transpile the ideal circuit to a circuit that can be directly executed by the backend
    qc = QuantumCircuit(num_qubits)
    transpiled_circuit = transpile(qc, backend)
    transpiled_circuit.draw('mpl')

    # Run the transpiled circuit using the simulated fake backend
    job = backend.run(transpiled_circuit)
    
    # counts = job.result().get_counts()
    # plot_histogram(counts)



    def edge_cost_fn(weight):
        # return math.exp(weight)
        return weight
    
    graph = rw.PyGraph()
    edges = connections.get_edges()
    weighted_edges = [(source, target, randrange(1,100)/1000) for source, target in edges]
    graph.extend_from_weighted_edge_list(weighted_edges)
    
    # paths = rw.graph_all_pairs_dijkstra_shortest_paths(graph, edge_cost_fn)

    min_max_length = float('inf')
    centroid = None
    for node in graph.node_indices():
        lengths = dijkstra_shortest_path_lengths(graph, node, edge_cost_fn)
        max_length = max(lengths.values())
        if max_length < min_max_length:
            min_max_length = max_length
            centroid = node

    print(f"Centroid: {centroid}")

    targeted = {}
    for i in range(num_qubits):
        targeted.update({i: 'n'})
    start = randrange(num_qubits)
    print("Start qubit: ", start)

    qc.h(start)
    # print(qc.draw())
    # mark this qubit as 'targeted'
    targeted[start] = 't'

    list_ctrl = []
    list_ctrl.append(start)
    # print("Initial list of control qubits: ", list_ctrl)
    # print("First element of the list: ", list_ctrl[0])

    def non_entangled_exist(target_dict):
        ret = False
        for i in range(len(target_dict)):
            if target_dict[i] == 'n':
                ret = True

        return ret

    iteration = 0
    while non_entangled_exist(targeted):
        iteration += 1
        # helper list_ctrl to be used for swap at the end of the loop
        swp_list_ctrl = []
        # for each node in list_ctrl find connected nodes which haven't been entangled (targeted)
        for i in list_ctrl:
            # find connected nodes in connections not targeted
            for j in connections:
                # apply cx on it
                # mark it as targeted
                # add it to swp_list_ctrl
                if j[0] == i:
                    if targeted[j[1]] == 'n':
                        qc.cx(i, j[1])
                        targeted[j[1]] = 't'
                        swp_list_ctrl.append(j[1])
                if j[1] == i:
                    if targeted[j[0]] == 'n':
                        qc.cx(i, j[0])
                        targeted[j[0]] = 't'
                        swp_list_ctrl.append(j[0])
        # clear list_ctrl
        list_ctrl = []
        # replace list_ctrl with swp_list_ctrl
        for i in swp_list_ctrl:
            list_ctrl.append(i)
        qc.barrier()
        

    return [qc.depth(), num_qubits]


    # graphviz_draw(graph,method='neato')
    graphviz_draw(rw.minimum_spanning_tree(graph,weight_fn=edge_cost_fn),method = 'dot')

# Testing the solution on the Sherbrooke fake backend with the starting qubit as the root of the spanning tree
print("The circuit depth: ", get_optimized_state_root(sherbrooke)[0])


Centroid: 64
Start qubit:  111
The circuit depth:  35


# Results

In [45]:
results = []

backends = [ prague, quitov2, auckland, sherbrooke ]

for backend in backends:
    result = get_optimized_state_root(backend)
    results.append(1-result[0]/result[1])

print(results)


Centroid: 12
Start qubit:  28
Centroid: 1
Start qubit:  0
Centroid: 8
Start qubit:  7
Centroid: 60
Start qubit:  114
[0.33333333333333337, 0.0, 0.4444444444444444, 0.7007874015748031]
